In [10]:
import os, random, torch
import numpy as np

def set_seed(seed):
    os.environ['PYTHONHASHSEED'] = str(seed) 
    random.seed(seed)  
    np.random.seed(seed)  
    torch.manual_seed(seed)  
    torch.cuda.manual_seed(seed)  
    torch.cuda.manual_seed_all(seed)  
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# 예시
set_seed(123)

In [11]:
######################################################
# (1) 라이브러리 임포트 및 기본 설정
######################################################
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split, Subset
from torchvision import datasets, transforms, models

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from tqdm import tqdm
from sklearn.metrics import confusion_matrix, classification_report

# 디바이스 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [12]:
######################################################
# (2) 하이퍼파라미터 & 경로 설정
######################################################
batch_size = 128
epochs = 20            # 예시로 20 epoch
learning_rate = 0.001
num_classes = 300

# 데이터셋 경로 (사용 환경에 맞게 수정)
train_path = "./dogun_convnext_tiny_try111/dl-bootcamp-2025-challenge_aug/train"
test_path  = "/test/final_exam/challenge/test"

# 체크포인트 저장 폴더
checkpoint_dir = "./checkpoint_efficientnet_b3"
os.makedirs(checkpoint_dir, exist_ok=True)  # 없으면 생성

In [13]:
######################################################
# (3) 데이터 변환 & 데이터셋/로더 정의
######################################################
# 논문 권장: EfficientNet-B3 → 300×300
transform_train = transforms.Compose([
    transforms.Resize((300, 300)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std =[0.229, 0.224, 0.225])
])

transform_test = transforms.Compose([
    transforms.Resize((300, 300)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std =[0.229, 0.224, 0.225])
])

# Train / Test 데이터셋
full_train_dataset = datasets.ImageFolder(train_path, transform=transform_train)
test_dataset       = datasets.ImageFolder(test_path,  transform=transform_test)

# Validation Split (예: 5%)
train_size = int(0.95 * len(full_train_dataset))
val_size   = len(full_train_dataset) - train_size
train_dataset, validation_dataset = random_split(full_train_dataset, [train_size, val_size])

# DataLoader
train_loader      = DataLoader(train_dataset,      batch_size=batch_size, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)
test_loader       = DataLoader(test_dataset,       batch_size=batch_size, shuffle=False)

print(f"Training samples:   {len(train_dataset)}")
print(f"Validation samples: {len(validation_dataset)}")
print(f"Test samples:       {len(test_dataset)}")

Training samples:   89190
Validation samples: 4695
Test samples:       4178


In [14]:
######################################################
# (4) 모델 정의 (EfficientNet-B3, scratch) & (선택)체크포인트 로드
######################################################
# 처음부터(scratch) → weights=None
effnet_b3 = models.efficientnet_b3(weights=None)

# 분류기 부분 수정 → 출력 차원 = 300개
in_features = effnet_b3.classifier[1].in_features  # 일반적으로 1536
effnet_b3.classifier[1] = nn.Linear(in_features, num_classes)
model = effnet_b3.to(device)

# [원하는 체크포인트 로드 시 주석 해제 예시]
# ckpt_path = os.path.join(checkpoint_dir, "checkpoint_epoch_14.pth")
# model.load_state_dict(torch.load(ckpt_path))
# print(f"Loaded checkpoint: {ckpt_path}")



In [15]:
######################################################
# (5) 옵티마이저, 스케줄러, 손실함수 정의
######################################################
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)


In [16]:
######################################################
# (6) 학습 함수, 검증 함수, 테스트 함수
######################################################
def train_model(num_epochs=epochs):
    for epoch in range(num_epochs):
        model.train()
        running_loss  = 0.0
        total_batches = len(train_loader)

        print(f"Epoch {epoch+1}/{num_epochs}")
        for batch_idx, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            if (batch_idx + 1) % 10 == 0 or (batch_idx + 1) == total_batches:
                print(f"  [Batch {batch_idx+1}/{total_batches}] Loss: {loss.item():.4f}")

        # Validation Loss
        val_loss = validate_model()

        # 매 Epoch마다 체크포인트 저장
        ckpt_path = os.path.join(checkpoint_dir, f"checkpoint_epoch_{epoch+1}.pth")
        torch.save(model.state_dict(), ckpt_path)
        print(f"Checkpoint saved: {ckpt_path}")

        # 스케줄러 step
        scheduler.step()

        print(f"==> Epoch [{epoch+1}/{num_epochs}] "
              f"Train Loss: {running_loss/total_batches:.4f} | "
              f"Val Loss: {val_loss:.4f} | "
              f"LR: {scheduler.get_last_lr()[0]:.6f}\n")

def validate_model():
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inputs, labels in validation_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
    return val_loss / len(validation_loader)

def test_model(model, data_loader):
    model.eval()
    correct    = 0
    total      = 0
    all_preds  = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)

            total   += labels.size(0)
            correct += (predicted == labels).sum().item()

            # F1, 혼동행렬 등을 위해 저장
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    acc = 100.0 * correct / total
    print(f"Test Accuracy: {acc:.2f}%")
    return all_labels, all_preds


In [ ]:
######################################################
# (7) 학습
######################################################
train_model(epochs)
print("----- Training Finished -----")


Epoch 1/20
  [Batch 10/697] Loss: 5.7658
  [Batch 20/697] Loss: 5.7318
  [Batch 30/697] Loss: 5.5755
  [Batch 40/697] Loss: 5.5357
  [Batch 50/697] Loss: 5.4891
  [Batch 60/697] Loss: 5.1448
  [Batch 70/697] Loss: 5.1922
  [Batch 80/697] Loss: 5.2258
  [Batch 90/697] Loss: 4.9997
  [Batch 100/697] Loss: 4.9275
  [Batch 110/697] Loss: 5.0622
  [Batch 120/697] Loss: 4.8077
  [Batch 130/697] Loss: 4.8134
  [Batch 140/697] Loss: 4.7954
  [Batch 150/697] Loss: 4.7897
  [Batch 160/697] Loss: 4.7874
  [Batch 170/697] Loss: 4.6618
  [Batch 180/697] Loss: 4.7769
  [Batch 190/697] Loss: 4.7537
  [Batch 200/697] Loss: 4.6750
  [Batch 210/697] Loss: 4.7512
  [Batch 220/697] Loss: 4.6122
  [Batch 230/697] Loss: 4.6075
  [Batch 240/697] Loss: 4.7130
  [Batch 250/697] Loss: 4.6648
  [Batch 260/697] Loss: 4.5440
  [Batch 270/697] Loss: 4.5439
  [Batch 280/697] Loss: 4.5187
  [Batch 290/697] Loss: 4.5154
  [Batch 300/697] Loss: 4.5043
  [Batch 310/697] Loss: 4.3692
  [Batch 320/697] Loss: 4.3583
  [Bat

In [ ]:
######################################################
# (8) 평가(Eval) & CSV 제출
######################################################
# (선택) 원하는 체크포인트 로드 - 주석 해제 시 사용
# custom_ckpt = os.path.join(checkpoint_dir, "checkpoint_epoch_14.pth")
# model.load_state_dict(torch.load(custom_ckpt))
# print(f"Loaded checkpoint: {custom_ckpt}")

print("[Step 8] Evaluating current model & Saving CSV...")
all_labels_main, all_preds_main = test_model(model, test_loader)

submission_main = pd.read_csv('./sample_submission.csv')
submission_main['Label'] = all_preds_main
submission_main.to_csv('./submission_epochMain.csv', index=False)
print("Submission file saved as 'submission_epochMain.csv'.")

In [ ]:
######################################################
# (9) 학습 이후 분석 (혼동행렬 & F1) + 낮은 F1 클래스 폴더로 복사
######################################################
cm = confusion_matrix(all_labels_main, all_preds_main)
plt.figure(figsize=(12, 10))
sns.heatmap(cm, cmap='Blues', annot=False, fmt='d')
plt.title("Confusion Matrix (EfficientNet-B3)")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.savefig("confusion_matrix.png", dpi=200)
plt.close()
print("Confusion matrix saved as 'confusion_matrix.png'")

class_names = test_dataset.classes
report_dict = classification_report(all_labels_main, all_preds_main,
                                    target_names=class_names,
                                    output_dict=True)
report_df = pd.DataFrame(report_dict).transpose()
report_df.to_csv("class_f1_report.csv", index=True)
print("Classification report (including F1) saved as 'class_f1_report.csv'")

class_report_only = report_df.iloc[:num_classes]
class_report_only = class_report_only.sort_values(by="f1-score", ascending=True)
lowest_30 = class_report_only.head(30)
lowest_30_path = "lowest_30_classes.csv"
lowest_30.to_csv(lowest_30_path)
print(f"Lowest 30 F1-score classes saved as '{lowest_30_path}'")

print("\n--- Lowest 30 Classes by F1-Score ---")
print(lowest_30[["precision", "recall", "f1-score"]])

import os
import shutil
from torchvision import datasets
from torch.utils.data import DataLoader

def save_low_f1_images_to_folder(df, dataset, out_root):
    low_class_names = df.index.tolist()
    c2i = dataset.class_to_idx
    i2c = {v: k for k, v in c2i.items()}
    os.makedirs(out_root, exist_ok=True)
    for path, idx in dataset.samples:
        if i2c[idx] in low_class_names:
            dst_dir = os.path.join(out_root, i2c[idx])
            os.makedirs(dst_dir, exist_ok=True)
            shutil.copy2(path, os.path.join(dst_dir, os.path.basename(path)))

save_low_f1_images_to_folder(lowest_30, full_train_dataset, "lowest_30_data")
print("\n[Step 9] Copied low-F1 class images to 'lowest_30_data' folder.")

In [ ]:
######################################################
# (10) F1 낮은 클래스 폴더로부터 파인튜닝
######################################################
def fine_tune_low_f1_classes_folder(
    data_root,
    transform_train,
    model,
    optimizer,
    scheduler,
    device,
    criterion,
    epochs,
    bs,
    ckpt_dir
):
    ds = datasets.ImageFolder(data_root, transform_train)
    dl = DataLoader(ds, batch_size=bs, shuffle=True)
    model.train()
    for e in range(epochs):
        total_loss = 0.0
        for x, y in dl:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            outputs = model(x)
            loss = criterion(outputs, y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        scheduler.step()
        save_path = os.path.join(ckpt_dir, f"fine_tune_folder_epoch_{e+1}.pth")
        torch.save(model.state_dict(), save_path)
        print(f"[Fine-Tune Folder] Epoch {e+1}/{epochs} | Loss: {total_loss/len(dl):.4f} | Saved: {save_path}")

print("[Step 10] Start fine-tuning on 'lowest_30_data'...")
fine_tune_low_f1_classes_folder(
    data_root="lowest_30_data",
    transform_train=transform_train,
    model=model,
    optimizer=optimizer,
    scheduler=scheduler,
    device=device,
    criterion=criterion,
    epochs=2,                # 예: 2 epoch
    bs=64,                   # 배치 사이즈
    ckpt_dir=checkpoint_dir
)
print("[Step 10] Fine-tuning finished.")

In [ ]:
######################################################
# (11) 최종 평가 (원하면 CSV 생성)
######################################################
print("[Step 11] Testing after fine-tuning...")
all_labels_ft, all_preds_ft = test_model(model, test_loader)

# 파인튜닝 후 CSV 저장
submission_ft = pd.read_csv('./sample_submission.csv')
submission_ft['Label'] = all_preds_ft
submission_ft.to_csv('./submission_epochFineTune.csv', index=False)
print("Submission file saved as 'submission_epochFineTune.csv'.")
print("----- Done -----")